# Eliza Chatbot
Project group 10 - Taylor Payne, Zachary Richardson, Timothy Slivka, Cody Boswell

The Eliza chatbot is one of the earliest examples of a natural lannguage program that was created in the 1960s by Joseph Weizenbaum (add citation). Eliza works by using simple pattern matching and substitution techniques that give the illusion that Eliza understands the user input. It spots keywords in user input and responds with predeined scripts or transforms statements into questions. 

For this assignment, Project Group 10 set out to mimix the functionality of the original Eliza chatbot to demostrate our understanding of regular expressions and basic natural language processing. By implementing word-spotting and statement trasnformation logic, we created a simple interactive chatbot that responds to user input in a way that simulates a supportive psychotherapist. This project allowed us to apply regex-based pattern matching and conversational flow design to explore how early NLP systems like Eliza were able to engage users using rul-based responses.

Our code begins by importing the required packages. The re package is utilized in order for Eliza to recognize regex patterns. Random is used to randomly select possible responses. The nltk package is used to parse user input and lemmatize the tokens to allow for more robust pattern recognition. 
We next set up our Eliza chatbot to handle pronoun transformations. We accomplished this by defining what pronouns needed to be transformed. We then defined the function transform_patterns. This function is used to transform I want, I feel, I am, and I don't know statements into questions while incorporating our pronoun transformations. For example, this would transform the statement 'I am sad' to 'Why are you sad?'. It should be noted that this transformation only occurs if a phrase defined in our pattern tuple isn't recognized. Our pattern tuple defines certain regex patterns and creates predefined responses relating to topics such as emotions, health, and identity. We then define fallback responses that occur when a word pattern is not recognized in user input. Our lemmatize_input function lemmatizes lemmatizes each word in user input- meaning that it reduces it down to its most basic form. This enables our regex patterns to be able to capture a wider range of user input. 

Project Group Ten added several "bonus features" to this chatbot. We began my adding a time-appropriate greeting using the function time_of_day_greeting. This function records the system time of the user and greets them with either good morning, good afternoon, or good evening. Our group next added a time delay built into the eliza_response function. This was in order to allow a sense of "realness" as if our Eliza was "thinking" of a response. The last bonus feature we added was through the function random_encouraging_statements. As the name suggests, this function allowed our Eliza bot to provide random encouraging statements 20% of the time.

In [45]:
import re # package required for regular expression recognition
import random # package to allow for random response selection
import nltk 
from nltk.stem import WordNetLemmatizer # package for lemmatizing user input
from nltk.tokenize import word_tokenize # package to tokenize user input
from nltk.corpus import wordnet # lexical database for the English language
import time # package used to introduce time delay of time-based greeting bonus features

### Pronoun Replacement Rules

Replacements is a list of regex-based substitution rules used to transform the user's input into a reflective response. The purpose of this is to make Eliza's responses sound more natural and conversational by mirroring what the user says from Eliza's POV. Each tuple in this list has a regex pattern that matches a specific word or phrase, such as I, and swaps it with a replacement string, such as you. This is utilized in the transform_patterns function in order to transform pronouns.

In [33]:
replacements = [
    (re.compile(r'\bI am\b', re.IGNORECASE), 'you are'),
    (re.compile(r'\bI\b', re.IGNORECASE), 'you'),
    (re.compile(r'\bme\b', re.IGNORECASE), 'you'),
    (re.compile(r'\bmy\b', re.IGNORECASE), 'your'),
    (re.compile(r'\bmine\b', re.IGNORECASE), 'yours'),
    (re.compile(r'\byou are\b', re.IGNORECASE), 'I am'),
    (re.compile(r'\byou\b', re.IGNORECASE), 'me'),
    (re.compile(r'\byour\b', re.IGNORECASE), 'my'),
    (re.compile(r'\byours\b', re.IGNORECASE), 'mine')
]

### Transform Statements into Questions

Transform_patterns defines a set of regex-based transformation rules used in an Eliza-style chatbot to convert user statements into questions. This allows our Eliza chatbot to sound thoughtful and empathetic. Each tuple starts with a regular expression to match certain sentence structures such as "I want" or "I feel". A lambda function then recieves a match object from the regex and the user's name and transforms if by applying replacements rules. A grammatically correct follow-up quetsion is then built based on the user's input.

In [34]:
transform_patterns = [
    (re.compile(r'\bi want to (.+)', re.IGNORECASE), 
     lambda match, name: f"{name}, why do you want to " + 
        (lambda text: [pattern.sub(replacement, text) for pattern, replacement in replacements][-1])(match.group(1)) + "?"),

    (re.compile(r'\bi feel (.+)', re.IGNORECASE),
     lambda match, name: f"What makes you feel " + 
        (lambda text: [pattern.sub(replacement, text) for pattern, replacement in replacements][-1])(match.group(1)) + "?"),

    (re.compile(r'\bi am (.+)', re.IGNORECASE),
     lambda match, name: f"How long have you been " + 
        (lambda text: [pattern.sub(replacement, text) for pattern, replacement in replacements][-1])(match.group(1)) + "?"),

    (re.compile(r"\bi don'?t know (.+)", re.IGNORECASE),
     lambda match, name: f"What makes you uncertain about " + 
        (lambda text: [pattern.sub(replacement, text) for pattern, replacement in replacements][-1])(match.group(1)) + "?"),
]

### Patterns to Recognize

The patterns variable is a list of dictionaries where each dictionary contains a regex and a corresponding response. This structure enables word spotting that allows Eliza to recognize certain keywords or phrases in user input and respond in a relevent way. Each dictionary has a pattern and a response. The pattern is a compiled regex pattern that matches emotional wrods, life circumstances, or common phrases. The response defines the text Eliza will response with if the pattern matches the user's input. This expansive list allows Eliza to detect mental health-related topics, suport relationship, work, school, and health-related triggers, handles existential questions, and addresses critical safety concerns. This list is scanned linearly, so the first match that is found is used. 

In [49]:
patterns = [
    # Emotions
    {'pattern': re.compile(r'\b(sad|depress|depressed|unhappy)\b', re.IGNORECASE), 'response': "I'm sorry to hear that. Can you talk more about it?"},
    {'pattern': re.compile(r'\b(happy|joyful|excited)\b', re.IGNORECASE), 'response': "That's wonderful to hear. What’s been making you feel this way?"},
    {'pattern': re.compile(r'\b(anxious|nervous|worried)\b', re.IGNORECASE), 'response': "What’s been causing you to feel anxious?"},
    {'pattern': re.compile(r'\b(tired|exhausted|drained)\b', re.IGNORECASE), 'response': "What’s been wearing you out lately?"},
    {'pattern': re.compile(r'\b(lonely|alone)\b', re.IGNORECASE), 'response': "Do you often feel this way?"},
    {'pattern': re.compile(r'\b(mad|angry|furious)\b', re.IGNORECASE), 'response': "What do you think is the source of your anger?"},
    {'pattern': re.compile(r'\b(confused|uncertain)\b', re.IGNORECASE), 'response': "What are you feeling uncertain about?"},
    {'pattern': re.compile(r'\b(relaxed|calm|peaceful)\b', re.IGNORECASE), 'response': "That’s a good state to be in. What helps you stay that way?"},
    {'pattern': re.compile(r'\b(frustrated|irritated|annoyed)\b', re.IGNORECASE), 'response': "Can you tell me more about what's been frustrating you?"},
    {'pattern': re.compile(r'\b(hopeless|worthless|numb)\b', re.IGNORECASE), 'response': "These feelings can be heavy. What do you think contributes to them?"},

    # Desires/Needs
    {'pattern': re.compile(r'\b(crave|craving)\b', re.IGNORECASE), 'response': "Why don't you tell me more about your cravings?"},
    {'pattern': re.compile(r'\b(need|needs)\b', re.IGNORECASE), 'response': "What makes you feel that you need this?"},
    {'pattern': re.compile(r'\b(want|wants)\b', re.IGNORECASE), 'response': "Why do you want that?"},
    {'pattern': re.compile(r'\b(dream|dreams)\b', re.IGNORECASE), 'response': "Tell me more about your dreams."},
    {'pattern': re.compile(r'\b(goal|goals|ambition)\b', re.IGNORECASE), 'response': "What are your goals, and how do you plan to reach them?"},

    # Social Relationships
    {'pattern': re.compile(r'\b(friend|friends)\b', re.IGNORECASE), 'response': "Do you feel supported by your friends?"},
    {'pattern': re.compile(r'\b(family|parents|sibling)\b', re.IGNORECASE), 'response': "How is your relationship with your family?"},
    {'pattern': re.compile(r'\b(partner|relationship)\b', re.IGNORECASE), 'response': "Tell me about your relationship."},
    {'pattern': re.compile(r'\b(relationship|dating)\b', re.IGNORECASE), 'response': "How do you feel about your current relationship status?"},
    {'pattern': re.compile(r'\b(divorce|breakup|ex)\b', re.IGNORECASE), 'response': "That can be tough. How are you coping with it?"},

    # Work and School
    {'pattern': re.compile(r'\b(job|work|boss)\b', re.IGNORECASE), 'response': "How do you feel about your work situation?"},
    {'pattern': re.compile(r'\b(school|study|class)\b', re.IGNORECASE), 'response': "How are things going with school?"},
    {'pattern': re.compile(r'\b(homework|assignment)\b', re.IGNORECASE), 'response': "Do you feel stressed by your schoolwork?"},
    {'pattern': re.compile(r'\b(stress|stressed)\b', re.IGNORECASE), 'response': "What’s been stressing you out lately?"},
    {'pattern': re.compile(r'\b(overwhelmed|burned out)\b', re.IGNORECASE), 'response': "It sounds like a lot. How do you usually cope with this?"},

    # Health
    {'pattern': re.compile(r'\b(sick|ill|unwell)\b', re.IGNORECASE), 'response': "I’m sorry to hear that. How long have you been feeling this way?"},
    {'pattern': re.compile(r'\b(pain|hurt|aches)\b', re.IGNORECASE), 'response': "Where are you feeling pain?"},
    {'pattern': re.compile(r'\b(sleep|insomnia)\b', re.IGNORECASE), 'response': "How has your sleep been lately?"},
    {'pattern': re.compile(r'\b(appetite|eating|food)\b', re.IGNORECASE), 'response': "Have you noticed any changes in your appetite?"},
    {'pattern': re.compile(r'\b(medication|therapy)\b', re.IGNORECASE), 'response': "Are you currently getting support for this?"},

    # Existential
    {'pattern': re.compile(r'\b(purpose|meaning|why am I)\b', re.IGNORECASE), 'response': "Do you often think about the meaning of life?"},
    {'pattern': re.compile(r'\b(regret|guilt|ashamed)\b', re.IGNORECASE), 'response': "Would you like to talk more about that feeling of regret?"},
    {'pattern': re.compile(r'\b(death|die|dying)\b', re.IGNORECASE), 'response': "That’s a difficult topic. What are your thoughts on it?"},
    {'pattern': re.compile(r'\b(faith|god|spiritual)\b', re.IGNORECASE), 'response': "How does your spirituality influence your thoughts?"},
    {'pattern': re.compile(r'\b(future|fear of the future)\b', re.IGNORECASE), 'response': "What are your biggest worries about the future?"},

    # Identity
    {'pattern': re.compile(r'\b(self-esteem|confidence|insecure)\b', re.IGNORECASE), 'response': "How do you feel about yourself?"},
    {'pattern': re.compile(r'\b(identity|who am I)\b', re.IGNORECASE), 'response': "Have you been questioning your identity lately?"},
    {'pattern': re.compile(r'\b(success|failure)\b', re.IGNORECASE), 'response': "What does success mean to you?"},
    {'pattern': re.compile(r'\b(perfectionist|expectations)\b', re.IGNORECASE), 'response': "Do you feel pressure to meet certain expectations?"},
    {'pattern': re.compile(r'\b(change|improve|grow)\b', re.IGNORECASE), 'response': "What kind of change are you hoping for?"},

    # Life
    {'pattern': re.compile(r'\b(phone|social media|internet)\b', re.IGNORECASE), 'response': "How does technology impact your mood or thoughts?"},
    {'pattern': re.compile(r'\b(addiction|compulsion)\b', re.IGNORECASE), 'response': "Are there behaviors you're trying to control or limit?"},
    {'pattern': re.compile(r'\b(habit|routine)\b', re.IGNORECASE), 'response': "How do your daily routines affect your mindset?"},
    {'pattern': re.compile(r'\b(hobby|interests)\b', re.IGNORECASE), 'response': "What activities bring you joy or calm?"},
    {'pattern': re.compile(r'\b(travel|vacation)\b', re.IGNORECASE), 'response': "Do you find travel relaxing or stressful?"},
    {'pattern': re.compile(r'\b(a long time|a while|months|years)\b', re.IGNORECASE), 'response': "How are you coping with these long-term feelings?"},

    # Eating Disorder
    {'pattern': re.compile(r'\b(bulimia|throw up)\b', re.IGNORECASE), 'response': "?"},
    {'pattern': re.compile(r'\b(starve myself|starving|anorexic)\b', re.IGNORECASE), 'response': "?"},

    # Suicidal/Harmful Thoughts
    {'pattern': re.compile(r'\b(hurt|harm|self harm|self-harm)\b', re.IGNORECASE), 'response': "Self harm is a very serious issue. What triggers cause you to harm yourself?"},
    {'pattern': re.compile(r'\b(suicide|suicidal|kill)\b', re.IGNORECASE), 'response': "Suicide is very serious. Please contact the suicide hotline by dialing 988."},

    # Response Patterns
    {'pattern': re.compile(r"\b(I don't know|not sure|no idea|I do not know)\b", re.IGNORECASE), 'response': "It's okay to not have all the answers. Can we explore that together?"},
    {'pattern': re.compile(r'\b(rather not say|prefer not to answer|don\'t want to talk)\b', re.IGNORECASE), 'response': "That’s perfectly fine. We can talk about something else when you're ready."},
    {'pattern': re.compile(r'\b(my boss|my parents|my partner|they made me)\b', re.IGNORECASE), 'response': "How do these people affect your feelings and choices?"},
    {'pattern': re.compile(r'\b(walk|exercise|sleep|music|meditate|therapy)\b', re.IGNORECASE), 'response': "It’s great that you have some coping tools. Do they help consistently?"},
    {'pattern': re.compile(r'\b(whatever|it doesn’t matter|who cares|I don’t care)\b', re.IGNORECASE), 'response': "It sounds like you may be feeling emotionally overwhelmed. Can you tell me more?"},
    {'pattern': re.compile(r'\b(better|improving|getting there|healing)\b', re.IGNORECASE), 'response': "That’s a positive step. What do you think has helped the most?"},
    {'pattern': re.compile(r'\b(my fault|I messed up|I’m to blame)\b', re.IGNORECASE), 'response': "What makes you feel responsible for this?"},
    {'pattern': re.compile(r'\b(scared|afraid|fear|nervous)\b', re.IGNORECASE), 'response': "Can we talk more about what makes you feel this way?"},
    {'pattern': re.compile(r'\b(numb|nothing|empty|disconnected)\b', re.IGNORECASE), 'response': "That sounds difficult. When did you start feeling like this?"},
    {'pattern': re.compile(r'\b(long|while)\b', re.IGNORECASE), 'response': "I'm sorry you've been dealing with this feeling for a long period of time. How do you cope with these feelings?"},
]

### Fallback Responses

Fallback_responses is a list of default replies that he Eliza chatbot uses when no transformation pattern or keyword matches the user's input. This ensures that Eliza still responds even if the input is unclear in order to maintain conversational flow. 

In [36]:
fallback_responses = [
    "Can you say that another way?",
    "I'm not sure I understand, but I'm here to listen.",
    "Tell me more about that.",
    "Please go on.",
]

### Lemmatize Input

This defines a lemmatization function which is used to simplify user input in a sentence to their base form. It begins by creating an isntance of NLTK's WordNetLemmatizer. lemmatize_input is then defined and begins by splitting the input sentence into individual words and then applied lemmatization to each word. It uses the defaul part of speech (noun), so it may not always lemmatize verbs or adjectives correctly. It then joins the list of lemmatized words back into a sentence as a single string. 

In [46]:
lemmatizer = WordNetLemmatizer()

def lemmatize_input(text):
    tokens = word_tokenize(text)
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

### Adds a Time Delay to Eliza Response

This code defines a function called eliza_response that simulates a human-like delay when the chatbot replied to the user. This function displays a message to the user indicating that Eliza is "typing" in order to mimick a human response. The program pauses for 1.5 seconds to mimick a human taking time to think/type. It will then print Eliza's response after this delay.  

In [37]:
def eliza_response(text, delay=1.5):
    print("[Eliza is typing...]")
    time.sleep(delay)  # Delay in seconds
    print(f"[Eliza] {text}")

### Random Encouraging Statements

This list of supportive messages and a function that randomly selects and prints one, simulating an encouraging therapist. This function creates a 20% probability that Eliza will say an encouraging statemnet after a normal reply. The reply is randomly selected form the encouraging_statements list. 

In [38]:
encouraging_statements = [
    "You're doing great by opening up.",
    "Thank you for sharing with me.",
    "I appreciate you opening up to me.",
    "I appreciate you sharing this with me."
]

def random_encouraging_statements(chance = 0.2):
    if random.random() < chance:
        eliza_response(f"{random.choice(encouraging_statements)}")

### Eliza Greeting Based on Time of Day

The time_of_day_greeting function is desined to return a greeting based on the current time of day. This adds a human-like touch to Eliza. It works by checking the current hour on the user's system and returns "Good morning" before 12pm, "Good afternoon" between 12pm and 5pm, and "Good evening" between 5pm and 12am. 

In [40]:
def time_of_day_greeting():
    time_of_day = time.strftime('%H:%M:%S')
    hour_of_day = int(time.strftime('%H'))

    if hour_of_day < 12:
        return 'Good morning'
    elif hour_of_day < 17:
        return 'Good afternoon'
    else:
        return 'Good evening'

### Chatbot Loop

This function is the core logic of our Eliza chatbot. It begins by calling the time_of_day function to gather the user's system hour and provide an appropriate greeting. It then prompts the user for their name and removes any extra whitespace around the input. Eliza replies, greeting the user by name using the eliza_response function, which includes the time delay and Eliza is typing feature. It then starts an infinite loop that runs until the user types exit, quit, or bye. It will ask the user for their next statement and remove extra spaces. If the user indicates that they want to end the session, Eliza responds and the loop breaks. The input is lemmatized. The input is checked to see if it fits any known sentence structures, such as "I want". If there is a match, the transform_patterns function is used to provide a realistic question transformation. It will then send this response and skip this in the next loop iteration. If there is no transformation match, it checks for keywords. If matches, Eliza responds with the pre-defined message. Eliza then has a 20% chance of providing a random encouraging statement. If no patterns or transformation match, Eliza picks a generic fallback. 

In [47]:
def main():

    time_of_day = time_of_day_greeting() # time of day function to provide time-appropriate greeting. 
    
    print("[Eliza] Hi, I'm a psychotherapist. What is your name?") # prompts user and asks for name
    patient_name = input("=> ").strip() # takes user input and strips white spaces
    eliza_response(f"Hi {patient_name}. How can I help you today?") # Eliza responds using patient name and asks what help the user needs

    while True: # starts infinite while loop
        user_input = input(f"[{patient_name}] ").strip() # takes user input and strips white spaces.
 
        if user_input.lower() in ['exit', 'quit', 'bye']: # checks if user indicates end of conversation
            eliza_response(f"Goodbye {patient_name}. Take care!")
            break

        lemmatized_input = lemmatize_input(user_input) # if user does not indicate end of converation, input is lemmatized


        # Check transformation patterns first
        for pattern, func in transform_patterns:
            match = pattern.search(lemmatized_input) # searches for pattern match in lemmatized input
            if match:
                eliza_response(func(match, patient_name)) # if there is a match, Eliza provides the corresponding response
                break
        else: 
            # Check for keyword spotting
            for entry in patterns:
                if entry['pattern'].search(user_input): # searches for pattern match in lemmatized input
                    eliza_response(entry['response']) # if there is a match, Eliza provides the corresponding response
                    random_encouraging_statements() # adds 20% of a random encouraging statement
                    break
            else:
                # Fallback response
                eliza_response(random.choice(fallback_responses)) # provides a fallback response if no matches

### Run Eliza

This line ensures that the main() function runs only when the script is exectued direclty, not when it's imported as a module into another program. This is a standard Python practice that allows this file to service both as a standalone program and a reusable module. This prompts Eliza to run.

In [48]:
if __name__ == "__main__":
    main()

[Eliza] Hi, I'm a psychotherapist. What is your name?


=>  Taylor


[Eliza is typing...]
[Eliza] Hi Taylor. How can I help you today?


[Taylor]  I need to talk to someone


[Eliza is typing...]
[Eliza] What makes you feel that you need this?


...

[Eliza is typing...]
[Eliza] It’s great that you have some coping tools. Do they help consistently?
[Eliza is typing...]
[Eliza] You're doing great by opening up.


[Taylor]  quir


[Eliza is typing...]
[Eliza] Can you say that another way?


[Taylor]  quit


[Eliza is typing...]
[Eliza] Goodbye Taylor. Take care!
